In [74]:
SAMPLE_RATE = 44100
ONSET_DURATION = 0.1

## ✨ Drum Mapping

In [75]:
# -- 드럼 매핑 데이터
code2drum = {0:'CC', 1:'HH', 2:'RC', 3:'ST', 4:'MT', 5:'SD', 6:'FT', 7:'KK'}
drum2code = {v: k for k, v in code2drum.items()}

In [76]:
# -- 1 long wav file 가져오기
# wav_root_path = '../data/tmp_wav_midi/'
# wav_file = wav_root_path + '1_funk-groove1_138_beat_4-4_1.wav'

# # -- onset 자른 파일 저장할 경로
# save_path = '../data/tmp_trimmed_wav_midi/'

In [77]:
# -- onset 나누기
# -- file name : 본래wavfile이름_몇번째onset인지.wav
import os
from essentia.standard import MonoLoader, OnsetDetection, Windowing, FFT, CartesianToPolar, FrameGenerator, Onsets, AudioOnsetsMarker, StereoMuxer, AudioWriter
from tempfile import TemporaryDirectory
import scipy.io.wavfile
from essentia.standard import *


# onset detection function
def onset_detect(input_path):
  # Load audio file.
  audio = MonoLoader(filename=input_path)()

  # 1. Compute the onset detection function (ODF).

  # The OnsetDetection algorithm provides various ODFs.
  od_hfc = OnsetDetection(method='hfc')
  od_complex = OnsetDetection(method='complex')

  # We need the auxilary algorithms to compute magnitude and phase.
  w = Windowing(type='hann')
  fft = FFT() # Outputs a complex FFT vector.
  c2p = CartesianToPolar() # Converts it into a pair of magnitude and phase vectors.

  # Compute both ODF frame by frame. Store results to a Pool.
  pool = essentia.Pool()
  for frame in FrameGenerator(audio, frameSize=1024, hopSize=512):
      magnitude, phase = c2p(fft(w(frame)))
      pool.add('odf.hfc', od_hfc(magnitude, phase))
      pool.add('odf.complex', od_complex(magnitude, phase))

  # 2. Detect onset locations.
  onsets = Onsets()

  onsets_hfc = onsets(# This algorithm expects a matrix, not a vector.
                      essentia.array([pool['odf.hfc']]),
                      # You need to specify weights, but if we use only one ODF
                      # it doesn't actually matter which weight to give it
                      [1])

  onsets_complex = onsets(essentia.array([pool['odf.complex']]), [1])

  # onset 개수가 짝수인 것을 우선
  if len(onsets_hfc) % 2 == 0:
    return onsets_hfc
  elif len(onsets_complex) % 2 == 0:
    return onsets_complex
  elif len(onsets_hfc) > len(onsets_complex):
    return onsets_hfc
  else:
    return onsets_complex
  
# trimmed audio
def audio_trim_per_onset(audio, onsets, sr=SAMPLE_RATE, duration=ONSET_DURATION):
  trimmed_audios = []
  for i in range(0, len(onsets)):
    start = (int)((onsets[i]) * sr)
    if i + 1 < len(onsets):
      end_by_onset = (int)(onsets[i + 1] * sr)
    end_by_duration = (int)((onsets[i] + duration) * sr)

    if i + 1 < len(onsets):
      end = min(end_by_onset, end_by_duration)
    else:
      end = end_by_duration

    trimmed = audio[start:end]
    trimmed_audios.append(trimmed)

  return trimmed_audios



# trimmed audio -> wav file write
def write_trimmed_audio(root_path, name, trimmed_audios):
  start = 1
  for audio in trimmed_audios:
    # exist or not
    if not os.path.exists(root_path):
      # if the demo_folder directory is not present
      # then create it.
      os.makedirs(root_path)
    scipy.io.wavfile.write(f'{root_path}/{name}_{start:04}.wav', SAMPLE_RATE, audio)
    start += 1

In [78]:
# -- midi 파일 까서 labeling
from music21 import *

def extract_midi_information(midi_file):
    midi_stream = converter.parse(midi_file)
    
    hits = []
    # 스트림 평탄화: 중첩된 스트림을 평탄화 한다.
    for element in midi_stream.flatten().notes:
        if isinstance(element, note.Unpitched):# -- unpitched : 한 개 침
            # -- None 없애기
            _ele = element.storedInstrument.percMapPitch
            # _ele = element.storedInstrument.instrumentName
            if _ele == None:
                # print("percussion 잡힘")
                return
            else:
                hits.append([_ele])
                print("Offset:", element.offset, "| Pitch:", element.pitch.midi, "| Duration:", element.duration.quarterLength)
        elif isinstance(element, percussion.PercussionChord):# -- percussion : 동시에 여러 개 침
            allNotes = []
            for thisNote in element.notes:
                if isinstance(thisNote, note.Unpitched):
                    # -- None 없애기
                    _ele = thisNote.storedInstrument.percMapPitch
                    if _ele == None:
                        # print("percussion 잡힘")
                        return
                    else:
                        allNotes.append(_ele)
                        print("Offset:", element.offset, "| Pitch:", element.pitch.midi, "| Duration:", element.duration.quarterLength)
            
                hits.append(allNotes)
    return hits

In [79]:
import os
import librosa

# load audio & trimmed per onset
# [pattern, per_drum]
# per_drum: [1_HH, 2_RC, ...], pattern: [P1, P2, ...]
# 04, 08, 16
def main(root_path, trim_path):
  result_data_set = []
  datas = os.listdir(root_path)
  
  # 1_funk-groove1_138_beat_4-4_1
  for d in datas:
    file = os.path.join(root_path, d)

    if d.endswith('.wav'):
      name = d[:-4]

      # trimming audioget
      audio, sr = librosa.load(file, sr=SAMPLE_RATE)
      # detect onsets
      onsets = onset_detect(file)
      trimmed_audios = audio_trim_per_onset(audio, onsets, sr)
      trim_len = len(trimmed_audios)

      midi_file = os.path.join(root_path, name+'.midi')
      print(midi_file)
      
      midi_len = 0
      if midi_file.endswith('.midi'):
        result = extract_midi_information(midi_file)
        if(result != None):
          result_data_set.append(result)
          midi_len = len(result)

      if midi_len == 0: # midi percussion 있었다면 패스
        continue
          
      # -- onset 길이, midi 길이 비교
      # -- offset 일치하는 경우, 한 음으로 뭉치기
      if midi_len == trim_len:
        print(name, ">>>>> 길이 같음 >>>>>", trim_len)
      else:
        print("길이 다름 ", 'trim:', trim_len, 'midi:', midi_len)

        
      # -- 한 음으로 뭉치고, 

    # elif d.endswith('.txt') == False:
    #   new_root_path = os.path.join(root_path, d)
    #   new_trim_path = os.path.join(trim_path, d)
    #   main(new_root_path, new_trim_path)
  print(result_data_set)
# 드럼 녹음 data path
root_path = "../data/tmp_wav_midi/"
trim_path = "../data/tmp_trimmed_wav_midi"
main(root_path, trim_path)

../data/tmp_wav_midi/9_soul-groove9_105_beat_4-4_1.midi


AttributeError: 'PercussionChord' object has no attribute 'pitch'

In [ ]:
# -- onset, labeling shape 맞는 지 확인